In [133]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split  
from sklearn.svm import SVC  
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score ,roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
from sklearn.decomposition import PCA
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [134]:
f = pd.read_csv("clean_data.csv")
f = f.set_index('encounter_id')
f.head()

,CCI,patient_nbr,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,...,glipizide-metformin_Steady,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,diabetesMed_No,diabetesMed_Yes,readmitted.1
encounter_id,,,,,,,,,,,,,,,,,,,,,
2278392,0,8222157,1,41,0,1,0,0,0,1,...,0,1,0,1,0,1,0,1,0,0
149190,0,55629189,3,59,0,18,0,0,0,9,...,0,1,0,1,0,1,0,0,1,0
64410,0,86047875,2,11,5,13,2,0,1,6,...,0,1,0,1,0,1,0,0,1,0
500364,0,82442376,2,44,1,16,0,0,0,7,...,0,1,0,1,0,1,0,0,1,0
16680,8,42519267,1,51,0,8,0,0,0,5,...,0,1,0,1,0,1,0,0,1,0


In [156]:
f=f.drop(['readmitted.1'],axis=1)
f.columns

Index(['CCI', 'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'change',
       ...
       'glipizide-metformin_No', 'glipizide-metformin_Steady',
       'glimepiride-pioglitazone_No', 'glimepiride-pioglitazone_Steady',
       'metformin-rosiglitazone_No', 'metformin-rosiglitazone_Steady',
       'metformin-pioglitazone_No', 'metformin-pioglitazone_Steady',
       'diabetesMed_No', 'diabetesMed_Yes'],
      dtype='object', length=162)

In [157]:
#Splitting attributes and features
if 'patient_nbr' in f.columns:
    f = f.drop(['patient_nbr'],axis = 1)
dataLabel=f['readmitted']
dataAttributes=f.drop(['readmitted'],axis=1)

In [158]:
#Splitting the data into training and testing(70-30)
X_train, X_test, y_train, y_test = train_test_split(dataAttributes,dataLabel,test_size=0.30,random_state=100)

In [159]:
#Algorithm 1 -RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 150,oob_score=True,max_features=0.02,max_depth=7,min_samples_leaf=10)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=0.02, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [160]:
scores = cross_validation.cross_val_score(
rf, X_train, y_train, cv=5)

In [175]:
scores.mean()
rf.oob_score_

0.8877674209669268

In [172]:
#Prediction on training and Testing Data
y_PredRandForest=rf.predict(X_test)
y_PredRandForest_train=rf.predict(X_train)


conf_matrix_train=confusion_matrix(y_train,y_PredRandForest_train)
print("Confusion Matrix for Random Forest on training dataset")
print(conf_matrix_train)

accuracy=accuracy_score(y_train, y_PredRandForest_train)
f1_Score=f1_score(y_train, y_PredRandForest_train, average='weighted')
print("Accuracy of RandomForest {}".format(accuracy*100))
print("F1 Score OF RandomForest  {}".format(f1_Score*100))
ra_train=roc_auc_score(y_train,y_PredRandForest_train)
print("ROC AUC Score for training {}".format(ra_train))
matrix_report=classification_report(y_train,y_PredRandForest_train) 
print("Report")
print(matrix_report)

Confusion Matrix for Random Forest on training dataset
[[63241     0]
 [ 7995     0]]
Accuracy of RandomForest 88.77674209669269
F1 Score OF RandomForest  83.49873877223528
ROC AUC Score for training 0.5
Report
             precision    recall  f1-score   support

          0       0.89      1.00      0.94     63241
          1       0.00      0.00      0.00      7995

avg / total       0.79      0.89      0.83     71236



In [173]:
conf_matrixRandForest=confusion_matrix(y_test,y_PredRandForest)
print("Confusion Matrix for Random Forest on Testing dataset ")
print(conf_matrixRandForest)

accuracy=accuracy_score(y_test, y_PredRandForest)
f1_Score=f1_score(y_test, y_PredRandForest, average='weighted')
print("Accuracy of RandomForest {}".format(accuracy*100))
print("F1 Score OF RandomForest {}".format(f1_Score*100))



ra_test=roc_auc_score(y_test,y_PredRandForest)

print("ROC AUC Score for testing {}".format(ra_test))
matrix_reportRandForest=classification_report(y_test,y_PredRandForest) 
print("Report")
print(matrix_reportRandForest)

Confusion Matrix for Random Forest on Testing dataset 
[[27168     0]
 [ 3362     0]]
Accuracy of RandomForest 88.98788077301015
F1 Score OF RandomForest 83.80265329270131
ROC AUC Score for testing 0.5
Report
             precision    recall  f1-score   support

          0       0.89      1.00      0.94     27168
          1       0.00      0.00      0.00      3362

avg / total       0.79      0.89      0.84     30530



In [163]:
roc_auc_score(y_train,y_PredRandForest_train)

0.5

In [164]:
#Feature Importance
pd.DataFrame({'Variable':X_train.columns,
              'Importance':rf.feature_importances_}).sort_values('Importance', ascending=False)

,Variable,Importance
7,number_inpatient,0.134367
57,discharge_disposition_id_22,0.078514
6,number_emergency,0.061602
37,discharge_disposition_id_1,0.056837
1,time_in_hospital,0.048527
8,number_diagnoses,0.044943
4,num_medications,0.038240
0,CCI,0.032333
47,discharge_disposition_id_11,0.028131
39,discharge_disposition_id_3,0.024715
